In [49]:
import pandas as pd
from scipy.spatial import distance
from scipy.cluster.hierarchy import linkage, cophenet
from scipy.stats import pearsonr, spearmanr, kendalltau
import numpy as np
from scipy import stats

In [19]:
# The clean raw fpkm without filtering
data_full = pd.read_csv('E:\\deeplearning\\Hepatocarcinomes\\TCGA\\processed_backup\\fpkm_final_raw_CCL3L1.csv','\t', index_col=0)
display(data_full.head(5))

,A4ND-01A,A3R2-01A,AAVY-01A,A3CJ-01A,A7ME-01A,AAUZ-01A,A8TG-01A,AACZ-01A,A116-01A,A3A3-01A,...,AADO-01A,A7S9-01A,A8TE-01A,A496-01A,A3A1-01A,AAVW-01A,A9GT-01A,A5DF-01A,A5UE-01A,A86L-01A
Gene Name,,,,,,,,,,,,,,,,,,,,,
A2M,88.221089,54.556726,44.235349,880.734277,230.322842,816.703134,141.338309,85.643762,70.594743,3228.576499,...,737.877208,682.540746,1112.199026,701.699788,350.988192,169.539064,132.809318,68.750776,13.047954,88.473934
ACVR1C,0.055671,0.087972,0.134440,0.056397,0.006433,0.251584,0.070873,0.013824,0.089073,0.160888,...,0.036667,0.082120,0.181159,0.181241,0.132371,0.034846,0.275388,0.087154,0.228851,0.234036
ADAM12,0.383264,0.021877,0.000000,0.008091,0.118482,0.043991,0.169738,0.083798,0.064079,0.100025,...,0.112351,0.033382,0.015017,0.017335,0.018991,0.342288,0.122481,0.951603,0.011614,0.036022
ADM,6.015631,2.519748,8.464564,17.053880,1.256800,6.081892,6.882283,4.120268,8.471828,5.287338,...,3.614977,2.386349,24.398716,1.076925,8.967692,1.045235,2.694580,6.277661,5.577421,1.498936
ADORA2A,1.195694,0.509299,2.505970,1.122596,1.999686,1.770068,0.551034,0.839757,1.478947,2.520129,...,4.617703,0.711106,3.306181,1.070061,0.818742,2.185467,2.935925,0.764492,0.573550,2.242258


In [10]:
print(data_full.shape)
print("Summary for each col/sample:")
display(data_full.describe())

(734, 340)
Summary for each col/sample:


,A4ND-01A,A3R2-01A,AAVY-01A,A3CJ-01A,A7ME-01A,AAUZ-01A,A8TG-01A,AACZ-01A,A116-01A,A3A3-01A,...,AADO-01A,A7S9-01A,A8TE-01A,A496-01A,A3A1-01A,AAVW-01A,A9GT-01A,A5DF-01A,A5UE-01A,A86L-01A
count,734.000000,734.000000,734.000000,734.000000,734.000000,734.000000,734.000000,734.000000,734.000000,734.000000,...,734.000000,734.000000,734.000000,734.000000,734.000000,734.000000,734.000000,734.000000,734.000000,734.000000
mean,17.050788,13.198650,38.389330,66.928453,21.658704,60.128237,17.128776,14.697700,30.788731,108.256417,...,70.797065,33.082513,29.567974,36.810395,38.724675,19.376934,48.388164,19.895510,12.402692,15.328559
std,155.423523,107.182187,288.208849,692.698429,212.360847,524.065969,121.884854,80.840822,241.130035,1315.622448,...,792.466548,371.611225,284.250849,467.801924,305.189218,154.986559,314.115394,164.472691,97.477357,129.285847
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.628036,0.117193,0.129818,0.215496,0.376555,0.503502,0.305048,0.373803,0.302798,0.334455,...,0.540670,0.212618,0.187308,0.076920,0.287214,0.515906,0.661383,0.379980,0.149042,0.171602
50%,2.547160,0.903711,1.683362,1.593534,1.847085,2.991493,1.571881,1.879253,1.699239,2.094809,...,2.415193,1.450392,1.479369,0.918648,1.532266,2.306996,3.347590,1.935884,0.866342,1.279125
75%,6.697865,4.921456,8.801102,7.776025,6.388487,12.138068,6.286541,6.384056,7.729505,10.608332,...,12.679862,6.106774,6.180991,4.843743,9.239817,6.936389,11.571390,7.316916,5.614694,6.420348
max,3614.948517,2203.664557,6125.545170,14747.772633,4151.674557,10783.539107,2888.079463,1607.708215,4768.034015,33347.372947,...,19438.941764,8263.493712,6462.723721,9767.275819,5579.907239,3006.311892,5982.825233,3487.009130,2083.815941,3182.265886


In [11]:
print("Summary for each row/gene:")
display(data_full.T.describe())

Summary for each row/gene:


Gene Name,A2M,ACVR1C,ADAM12,ADM,ADORA2A,AKT1,ALDOA,ALDOC,ANGPT1,ANGPT2,...,WNT4,WNT5A,WNT5B,WNT7B,XCL1,ZAP70,ZC3H12A,ZEB1,ZEB2,CCL3L1
count,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,...,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000
mean,477.804679,0.162126,0.343209,6.336935,1.435997,10.090481,69.285765,27.279090,0.382958,1.041386,...,1.559349,1.177586,1.732296,0.185387,0.488021,0.776897,4.456345,1.815628,0.517271,1.069304
std,685.965612,0.195546,1.789819,9.109531,1.052622,5.549442,94.206415,31.298099,0.380192,1.085154,...,3.455290,1.657025,3.139799,1.226196,0.793923,1.155819,4.960505,0.969947,0.627846,2.892823
min,8.063961,0.000000,0.000000,0.641932,0.094151,1.596935,7.803906,0.149045,0.000000,0.069319,...,0.000000,0.000000,0.006880,0.000000,0.000000,0.015525,0.391197,0.377950,0.056094,0.000000
25%,91.047779,0.036511,0.027462,2.279152,0.776934,6.279743,28.567629,6.600658,0.128935,0.407516,...,0.042986,0.171825,0.173976,0.000000,0.077333,0.180697,1.636205,1.135161,0.222516,0.163454
50%,251.210728,0.088117,0.064803,3.806960,1.129963,8.935641,46.130439,16.662141,0.281967,0.724156,...,0.259997,0.695436,0.662919,0.009356,0.196791,0.406109,2.684488,1.691145,0.354273,0.456370
75%,525.235249,0.226706,0.184408,6.486066,1.775684,12.995687,78.459471,35.036506,0.525633,1.261851,...,1.381246,1.482879,1.731090,0.038669,0.581974,0.901772,5.120123,2.249201,0.612979,0.962755
max,5167.410193,1.263938,25.872119,103.768361,8.900005,41.633061,965.402045,185.567247,3.237895,8.359021,...,24.041550,15.615770,22.036023,20.405032,6.869837,12.190879,38.911911,6.666682,7.559379,44.230049


In [16]:
# Data transformation prepared for gene clustering
data = 1 + data_full
data = np.log2(data)
data_full_transformed_genecl = pd.DataFrame(stats.zscore(data+1.e-17, axis=0), columns=data_full.columns, index=data_full.index)

In [20]:
display(data_full_transformed_genecl.head(5))

,A4ND-01A,A3R2-01A,AAVY-01A,A3CJ-01A,A7ME-01A,AAUZ-01A,A8TG-01A,AACZ-01A,A116-01A,A3A3-01A,...,AADO-01A,A7S9-01A,A8TE-01A,A496-01A,A3A1-01A,AAVW-01A,A9GT-01A,A5DF-01A,A5UE-01A,A86L-01A
Gene Name,,,,,,,,,,,,,,,,,,,,,
A2M,2.587276,2.355819,1.537161,3.409093,3.230708,3.097641,2.811238,2.509429,2.003416,3.824636,...,3.035517,3.807068,4.177932,3.927295,2.903661,2.941036,2.040472,2.212304,1.245843,2.549520
ACVR1C,-1.171927,-0.820547,-0.837728,-0.870621,-1.061673,-0.931009,-0.960316,-1.068889,-0.924738,-0.876596,...,-1.038055,-0.895584,-0.818021,-0.722873,-0.890363,-1.109344,-0.988933,-1.006987,-0.754668,-0.783390
ADAM12,-0.942941,-0.871162,-0.919006,-0.900399,-0.978342,-1.043725,-0.892204,-1.015198,-0.940980,-0.908518,...,-0.994359,-0.929193,-0.928606,-0.831608,-0.960105,-0.902947,-1.072077,-0.554355,-0.914397,-0.919473
ADM,0.432736,0.127637,0.529196,0.935222,-0.424190,0.146140,0.579352,0.233922,0.588411,0.124806,...,-0.112048,-0.063602,1.420221,-0.312096,0.547471,-0.568792,-0.296474,0.463844,0.622769,-0.234416
ADORA2A,-0.551466,-0.556194,-0.110695,-0.426698,-0.199554,-0.437247,-0.674582,-0.589506,-0.349344,-0.219026,...,0.009875,-0.561416,0.125622,-0.314505,-0.577131,-0.217221,-0.255277,-0.632326,-0.551648,-0.031807


In [18]:
print(data_full_transformed_genecl.shape)
print("Summary for each col/sample:")
display(data_full_transformed_genecl.describe())

(734, 340)
Summary for each col/sample:


,A4ND-01A,A3R2-01A,AAVY-01A,A3CJ-01A,A7ME-01A,AAUZ-01A,A8TG-01A,AACZ-01A,A116-01A,A3A3-01A,...,AADO-01A,A7S9-01A,A8TE-01A,A496-01A,A3A1-01A,AAVW-01A,A9GT-01A,A5DF-01A,A5UE-01A,A86L-01A
count,7.340000e+02,7.340000e+02,7.340000e+02,7.340000e+02,7.340000e+02,7.340000e+02,7.340000e+02,734.000000,7.340000e+02,7.340000e+02,...,7.340000e+02,7.340000e+02,7.340000e+02,7.340000e+02,7.340000e+02,7.340000e+02,7.340000e+02,7.340000e+02,7.340000e+02,7.340000e+02
mean,-1.936084e-17,3.872167e-17,-1.887682e-16,9.680419e-17,3.872167e-17,-5.808251e-17,1.452063e-17,0.000000,7.744335e-17,-5.324230e-17,...,4.840209e-17,1.355259e-16,-1.936084e-17,-1.452063e-16,2.904126e-17,-1.790877e-16,-5.808251e-17,1.839280e-16,8.228356e-17,-3.388147e-17
std,1.000682e+00,1.000682e+00,1.000682e+00,1.000682e+00,1.000682e+00,1.000682e+00,1.000682e+00,1.000682,1.000682e+00,1.000682e+00,...,1.000682e+00,1.000682e+00,1.000682e+00,1.000682e+00,1.000682e+00,1.000682e+00,1.000682e+00,1.000682e+00,1.000682e+00,1.000682e+00
min,-1.217828e+00,-8.886394e-01,-9.190055e-01,-9.055257e-01,-1.066735e+00,-1.070481e+00,-1.013132e+00,-1.079934,-9.844298e-01,-9.650286e-01,...,-1.060385e+00,-9.531405e-01,-9.394799e-01,-8.441179e-01,-9.725416e-01,-1.136522e+00,-1.147299e+00,-1.071633e+00,-9.238783e-01,-9.470067e-01
25%,-8.048991e-01,-7.991419e-01,-8.403593e-01,-7.813713e-01,-8.144499e-01,-8.170336e-01,-8.077755e-01,-0.824451,-7.993857e-01,-7.940017e-01,...,-7.923636e-01,-8.125491e-01,-8.142335e-01,-7.901770e-01,-8.056369e-01,-8.064323e-01,-8.167975e-01,-8.224758e-01,-8.098053e-01,-8.237851e-01
50%,-1.450851e-01,-3.687024e-01,-2.829976e-01,-2.992218e-01,-2.407786e-01,-2.102163e-01,-2.845130e-01,-0.229189,-2.897859e-01,-2.953582e-01,...,-2.987462e-01,-2.995743e-01,-2.770944e-01,-3.697952e-01,-3.583395e-01,-1.875128e-01,-1.905143e-01,-2.384452e-01,-4.115322e-01,-3.060575e-01
75%,5.113620e-01,5.477407e-01,5.517090e-01,4.763087e-01,5.120389e-01,5.302118e-01,5.187346e-01,0.528447,5.313146e-01,4.882848e-01,...,5.617733e-01,4.770066e-01,4.986761e-01,4.409143e-01,5.652732e-01,5.070956e-01,5.007603e-01,5.671056e-01,6.274077e-01,6.123845e-01
max,5.723792e+00,5.328538e+00,4.699935e+00,5.201276e+00,5.510304e+00,4.700732e+00,5.133290e+00,4.859577,4.940792e+00,5.208557e+00,...,5.063245e+00,5.624749e+00,5.461093e+00,5.843134e+00,4.730512e+00,5.218135e+00,4.514690e+00,5.238805e+00,5.351278e+00,5.328502e+00


In [22]:
print("Summary for each row/gene:")
display(data_full_transformed_genecl.T.describe())

Summary for each row/gene:


Gene Name,A2M,ACVR1C,ADAM12,ADM,ADORA2A,AKT1,ALDOA,ALDOC,ANGPT1,ANGPT2,...,WNT4,WNT5A,WNT5B,WNT7B,XCL1,ZAP70,ZC3H12A,ZEB1,ZEB2,CCL3L1
count,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,...,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000,340.000000
mean,2.999547,-0.900481,-0.875625,0.220677,-0.399981,0.688155,1.868532,1.033809,-0.779557,-0.543694,...,-0.590189,-0.544480,-0.497321,-0.932633,-0.771364,-0.665176,0.053771,-0.273989,-0.724685,-0.637699
std,0.850434,0.145547,0.235381,0.471639,0.203851,0.276335,0.500496,0.804331,0.171855,0.284611,...,0.501922,0.399159,0.450940,0.233374,0.233785,0.216270,0.405144,0.238779,0.137731,0.306389
min,0.898012,-1.294987,-1.243451,-0.756557,-0.871207,-0.136462,0.891461,-0.846773,-1.161356,-1.152113,...,-1.243582,-1.245748,-1.097574,-1.293260,-1.298832,-1.030652,-0.726402,-0.837602,-0.959437,-1.239026
25%,2.420083,-1.004624,-0.976092,-0.121760,-0.546639,0.512757,1.532692,0.513314,-0.891494,-0.757361,...,-0.907660,-0.861409,-0.845971,-1.043182,-0.907882,-0.803137,-0.228083,-0.446085,-0.811163,-0.828769
50%,3.097911,-0.899132,-0.908508,0.141317,-0.441829,0.701180,1.793318,1.159522,-0.790943,-0.577432,...,-0.812232,-0.597861,-0.618215,-0.956754,-0.837715,-0.733668,-0.008515,-0.291002,-0.743967,-0.716946
75%,3.593170,-0.813600,-0.844598,0.474771,-0.296149,0.850619,2.143434,1.596254,-0.692207,-0.404060,...,-0.391532,-0.333365,-0.269560,-0.885660,-0.719952,-0.598948,0.309641,-0.120030,-0.668172,-0.529643
max,5.308802,-0.372315,1.055939,1.674201,0.480138,1.887467,3.758704,2.851461,-0.153299,0.310543,...,1.948930,1.100277,1.016907,0.913980,0.187327,0.363587,1.346238,0.519596,0.446293,0.991809


In [29]:
# Export transformed data for gene clustering
data_full_transformed_genecl.to_csv('E:\\deeplearning\\Hepatocarcinomes\\TCGA\\processed_backup\\fpkm_final_log2-zscore_CCL3L1_genecl.csv', index=True, sep='\t')

In [23]:
# The transformed fpkm for sample clustering, without filtering
data_full_transformed_samplecl = pd.read_csv('E:\\deeplearning\\Hepatocarcinomes\\TCGA\\processed_backup\\fpkm_final_log2-zscore_CCL3L1.csv','\t', index_col=0)

In [24]:
display(data_full_transformed_samplecl.head(5))

,A4ND-01A,A3R2-01A,AAVY-01A,A3CJ-01A,A7ME-01A,AAUZ-01A,A8TG-01A,AACZ-01A,A116-01A,A3A3-01A,...,AADO-01A,A7S9-01A,A8TE-01A,A496-01A,A3A1-01A,AAVW-01A,A9GT-01A,A5DF-01A,A5UE-01A,A86L-01A
Gene Name,,,,,,,,,,,,,,,,,,,,,
A2M,-0.736032,-1.106771,-1.267620,1.056780,0.009570,0.997777,-0.370477,-0.758973,-0.908284,2.072793,...,0.918444,0.857520,1.239211,0.879155,0.338102,-0.229012,-0.418836,-0.928705,-2.182820,-0.733817
ACVR1C,-0.584371,-0.376119,-0.087118,-0.579618,-0.914412,0.591915,-0.485577,-0.863851,-0.369124,0.072124,...,-0.709895,-0.413380,0.191740,0.192220,-0.099736,-0.722042,0.722096,-0.381315,0.465253,0.494350
ADAM12,0.472566,-0.436172,-0.501119,-0.476934,-0.165080,-0.371921,-0.030610,-0.259617,-0.314722,-0.215018,...,-0.181578,-0.402573,-0.456387,-0.449540,-0.444659,0.382324,-0.154371,1.505552,-0.466463,-0.394915
ADM,0.390183,-0.588001,0.814804,1.730666,-1.218295,0.403515,0.555368,-0.056446,0.815892,0.234748,...,-0.203794,-0.642795,2.214742,-1.336088,0.888257,-1.357893,-0.519252,0.442186,0.298715,-1.073759
ADORA2A,-0.106720,-1.192840,1.249200,-0.204823,0.797314,0.566572,-1.113808,-0.619177,0.244844,1.260878,...,2.615223,-0.829225,1.844871,-0.277439,-0.652466,0.971425,1.584374,-0.740207,-1.072048,1.022627


In [25]:
print(data_full_transformed_samplecl.shape)
print("Summary for each col/sample:")
display(data_full_transformed_samplecl.describe())

(734, 340)
Summary for each col/sample:


,A4ND-01A,A3R2-01A,AAVY-01A,A3CJ-01A,A7ME-01A,AAUZ-01A,A8TG-01A,AACZ-01A,A116-01A,A3A3-01A,...,AADO-01A,A7S9-01A,A8TE-01A,A496-01A,A3A1-01A,AAVW-01A,A9GT-01A,A5DF-01A,A5UE-01A,A86L-01A
count,734.000000,734.000000,734.000000,734.000000,734.000000,734.000000,734.000000,734.000000,734.000000,734.000000,...,734.000000,734.000000,734.000000,734.000000,734.000000,734.000000,734.000000,734.000000,734.000000,734.000000
mean,0.200085,-0.483601,0.098644,0.007284,-0.042124,0.498674,-0.076569,0.074380,-0.002780,0.341794,...,0.521803,-0.129778,-0.127558,-0.370163,0.094506,0.169874,0.713588,0.029694,-0.423200,-0.215894
std,0.779964,0.582294,0.956393,0.894212,0.662956,0.827607,0.777117,1.069574,0.903585,1.006003,...,1.098908,0.806739,0.923344,0.819030,0.840991,0.802937,0.945582,0.727830,0.702690,0.926820
min,-1.715635,-2.421219,-1.680797,-3.069008,-2.036707,-2.388113,-1.948322,-2.666324,-2.184676,-1.976439,...,-2.247292,-2.219383,-2.681006,-2.187206,-2.017750,-1.414793,-1.822218,-1.644972,-2.182820,-2.274296
25%,-0.327254,-0.831474,-0.618319,-0.553882,-0.505501,-0.154306,-0.562001,-0.709448,-0.538303,-0.391575,...,-0.338528,-0.609233,-0.713548,-0.883262,-0.471850,-0.410740,-0.044440,-0.424720,-0.854725,-0.772049
50%,0.143676,-0.552810,-0.212026,-0.230506,-0.084691,0.520732,-0.236151,-0.091159,-0.165383,0.084128,...,0.396567,-0.264602,-0.336688,-0.500070,-0.006600,0.029365,0.669517,-0.120209,-0.539587,-0.393001
75%,0.634470,-0.197533,0.798433,0.458852,0.335075,1.037847,0.305182,0.721809,0.369557,0.932353,...,1.294892,0.254767,0.288366,0.027711,0.635955,0.573212,1.354644,0.378971,-0.164189,0.195816
max,6.612338,3.129266,3.837604,9.118940,2.505036,4.913037,5.127282,8.615264,8.795769,4.943362,...,4.942437,4.822822,5.051094,5.919046,4.134104,3.563367,5.460592,5.078688,3.417172,5.981458


In [26]:
print("Summary for each row/gene:")
display(data_full_transformed_samplecl.T.describe())

Summary for each row/gene:


Gene Name,A2M,ACVR1C,ADAM12,ADM,ADORA2A,AKT1,ALDOA,ALDOC,ANGPT1,ANGPT2,...,WNT4,WNT5A,WNT5B,WNT7B,XCL1,ZAP70,ZC3H12A,ZEB1,ZEB2,CCL3L1
count,3.400000e+02,3.400000e+02,3.400000e+02,3.400000e+02,3.400000e+02,3.400000e+02,3.400000e+02,3.400000e+02,3.400000e+02,3.400000e+02,...,3.400000e+02,3.400000e+02,3.400000e+02,3.400000e+02,3.400000e+02,3.400000e+02,3.400000e+02,3.400000e+02,3.400000e+02,3.400000e+02
mean,5.851528e-16,4.741305e-16,-8.359326e-17,4.545384e-16,-8.150343e-16,-8.777293e-16,-7.236042e-16,-1.546475e-15,-2.089832e-17,1.802480e-16,...,4.284155e-16,2.037586e-16,-9.926700e-17,1.724111e-16,5.825406e-16,-8.633617e-16,8.725047e-16,2.539145e-15,7.836868e-16,1.149407e-16
std,1.001474e+00,1.001474e+00,1.001474e+00,1.001474e+00,1.001474e+00,1.001474e+00,1.001474e+00,1.001474e+00,1.001474e+00,1.001474e+00,...,1.001474e+00,1.001474e+00,1.001474e+00,1.001474e+00,1.001474e+00,1.001474e+00,1.001474e+00,1.001474e+00,1.001474e+00,1.001474e+00
min,-2.525743e+00,-9.587209e-01,-5.011185e-01,-1.669378e+00,-2.124857e+00,-2.924549e+00,-2.245452e+00,-2.305184e+00,-1.312955e+00,-1.428160e+00,...,-7.893586e-01,-1.179809e+00,-1.036076e+00,-3.033867e-01,-8.882116e-01,-1.091742e+00,-1.721359e+00,-2.084955e+00,-1.225016e+00,-9.848148e-01
25%,-7.116382e-01,-7.109339e-01,-4.198131e-01,-6.884150e-01,-7.198487e-01,-6.819678e-01,-6.625404e-01,-6.536939e-01,-7.747373e-01,-7.229993e-01,...,-7.309542e-01,-8.755320e-01,-8.073395e-01,-3.033867e-01,-6.794178e-01,-7.266005e-01,-7.371134e-01,-7.064024e-01,-6.628568e-01,-6.924397e-01
50%,7.718016e-02,-3.751933e-01,-3.126806e-01,-1.459955e-01,-1.948011e-01,-5.275627e-03,-5.337283e-02,8.336264e-02,-2.105654e-01,-2.022949e-01,...,-4.686577e-01,-1.667083e-01,-2.886239e-01,-2.702369e-01,-3.846638e-01,-3.032563e-01,-2.215866e-01,2.206086e-02,-2.696486e-01,-2.587891e-01
75%,6.526143e-01,4.531837e-01,6.788630e-03,4.822153e-01,5.724229e-01,7.401561e-01,6.291105e-01,7.067006e-01,5.617168e-01,4.942572e-01,...,4.145785e-01,5.653394e-01,4.504995e-01,-1.683333e-01,3.974607e-01,4.283820e-01,5.597744e-01,6.152340e-01,4.019280e-01,3.175210e-01
max,2.440793e+00,4.687339e+00,9.375677e+00,4.224364e+00,4.256958e+00,3.163572e+00,3.893404e+00,2.144005e+00,5.095878e+00,4.138525e+00,...,3.679727e+00,4.213177e+00,3.627258e+00,1.060197e+01,4.894577e+00,5.121068e+00,3.447217e+00,3.317536e+00,6.813420e+00,6.376650e+00


In [30]:
dist = distance.pdist(data_full_transformed_samplecl.T.values, lambda u, v: 1-pearsonr(u, v)[0])
# print(distance.pdist(data.T.values, metric="euclidean"))
print(cophenet(average(dist), dist)[0])

0.5264338015314423


In [62]:
linkages = ["single", "average", "complete"]
distances=["euclidean", "sqeuclidean", "cityblock", "cosine", "chebyshev", "braycurtis",
           "pearson", "spearman", "kendalltau"]
# Here "correlation" refers to pearson correlation distance
print("col/sample cluster:")
df_samplecl = pd.DataFrame(columns=distances)
for link in linkages:
    print(link)
    list_cophenet = []
    for dis in distances:
        if dis == "pearson":
            dist = distance.pdist(data_full_transformed_samplecl.T.values, lambda u, v: 1-pearsonr(u, v)[0])
        elif dis == "spearman":
            dist = distance.pdist(data_full_transformed_samplecl.T.values, lambda u, v: 1-spearmanr(u, v)[0])
        elif dis == "kendalltau":
            dist = distance.pdist(data_full_transformed_samplecl.T.values, lambda u, v: 1-kendalltau(u, v)[0])
        else:
            dist = distance.pdist(data_full_transformed_samplecl.T.values, metric=dis) # sqeuclidean
        cophenet_dist = cophenet(linkage(y=dist, method=link), dist)[0]
        list_cophenet.append(cophenet_dist)
        print(dis, cophenet_dist) # the best is chebyshev 0.90
    df_samplecl = pd.concat([df_samplecl, pd.DataFrame(list_cophenet, columns=[link], index=distances).T])
    print()
print("ward")
dist = distance.pdist(data_full_transformed_samplecl.T.values, metric="euclidean")
cophenet_dist = cophenet(linkage(dist, "ward"), dist)[0]
print("euclidean", cophenet_dist)
df_samplecl = pd.concat([df_samplecl, pd.DataFrame([cophenet_dist], columns=["euclidean"], index=["ward"])])
display(df_samplecl)

col/sample cluster:
single
euclidean 0.8045566470491374
sqeuclidean 0.802212701437113
cityblock 0.7664995887421667
cosine 0.28019596170133915
chebyshev 0.9215785683955344
braycurtis 0.32328474479713865
pearson 0.08094739755297073
spearman 0.062337117987069236
kendalltau 0.06144518587816047

average
euclidean 0.8559794791771173
sqeuclidean 0.8467913156665122
cityblock 0.8036327055495089
cosine 0.653139612806414
chebyshev 0.9602329417303489
braycurtis 0.657708298278698
pearson 0.5264338015314423
spearman 0.5444067507160023
kendalltau 0.5567383778514532

complete
euclidean 0.608083170998329
sqeuclidean 0.5985121985264147
cityblock 0.6751086089993145
cosine 0.5833470073014084
chebyshev 0.9071669624131886
braycurtis 0.5989273355205347
pearson 0.4740187297674706
spearman 0.5217771433873564
kendalltau 0.5048579371346407

ward
euclidean 0.32589495786046857


C:\Program Files\Anaconda3\envs\r-tensorflow\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,braycurtis,chebyshev,cityblock,cosine,euclidean,kendalltau,pearson,spearman,sqeuclidean
single,0.323285,0.921579,0.766500,0.280196,0.804557,0.061445,0.080947,0.062337,0.802213
average,0.657708,0.960233,0.803633,0.653140,0.855979,0.556738,0.526434,0.544407,0.846791
complete,0.598927,0.907167,0.675109,0.583347,0.608083,0.504858,0.474019,0.521777,0.598512
ward,NaN,NaN,NaN,NaN,0.325895,NaN,NaN,NaN,NaN


In [59]:
df_samplecl.to_csv('E:\\deeplearning\\Hepatocarcinomes\\TCGA\\processed_backup\\cophenet_samplecl.csv', index=True, sep='\t')

In [64]:
print("row/gene cluster:")
df_genecl = pd.DataFrame(columns=distances)
for link in linkages:
    print(link)
    list_cophenet = []
    for dis in distances:
        if dis == "pearson":
            dist = distance.pdist(data_full_transformed_genecl.values, lambda u, v: 1-pearsonr(u, v)[0])
        elif dis == "spearman":
            dist = distance.pdist(data_full_transformed_genecl.values, lambda u, v: 1-spearmanr(u, v)[0])
        elif dis == "kendalltau":
            dist = distance.pdist(data_full_transformed_genecl.values, lambda u, v: 1-kendalltau(u, v)[0])
        else:
            dist = distance.pdist(data_full_transformed_genecl.values, metric=dis) # sqeuclidean
        cophenet_dist = cophenet(linkage(y=dist, method=link), dist)[0]
        list_cophenet.append(cophenet_dist)
        print(dis, cophenet_dist) # the best is chebyshev 0.90
    df_genecl = pd.concat([df_genecl, pd.DataFrame(list_cophenet, columns=[link], index=distances).T])
    print()
print("ward")
dist = distance.pdist(data_full_transformed_genecl.values, metric="euclidean")
cophenet_dist = cophenet(linkage(dist, "ward"), dist)[0]
print("euclidean", cophenet_dist)
df_genecl = pd.concat([df_genecl, pd.DataFrame([cophenet_dist], columns=["euclidean"], index=["ward"])])
display(df_genecl)

row/gene cluster:
single
euclidean 0.6264072506472996
sqeuclidean 0.5448592982997105
cityblock 0.6211785652963931
cosine 0.22166881085256468
chebyshev 0.8056012921254104
braycurtis 0.2443165458742929
pearson 0.09384823410112272
spearman 0.16978039545894666
kendalltau 0.24088272540564987

average
euclidean 0.8641978211198202
sqeuclidean 0.8501846549952612
cityblock 0.8529036931653252
cosine 0.9270252375549798
chebyshev 0.871488285915703
braycurtis 0.6938049079487367
pearson 0.6752681550406535
spearman 0.6921532609314212
kendalltau 0.7122405326843573

complete
euclidean 0.8193720366280619
sqeuclidean 0.7447641748330516
cityblock 0.8306566293617713
cosine 0.8785483630420194
chebyshev 0.8246796271761563
braycurtis 0.6946359642583742
pearson 0.5455803047456254
spearman 0.6197554729903844
kendalltau 0.6113209281241403

ward
euclidean 0.5668969295744599


C:\Program Files\Anaconda3\envs\r-tensorflow\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,braycurtis,chebyshev,cityblock,cosine,euclidean,kendalltau,pearson,spearman,sqeuclidean
single,0.244317,0.805601,0.621179,0.221669,0.626407,0.240883,0.093848,0.169780,0.544859
average,0.693805,0.871488,0.852904,0.927025,0.864198,0.712241,0.675268,0.692153,0.850185
complete,0.694636,0.824680,0.830657,0.878548,0.819372,0.611321,0.545580,0.619755,0.744764
ward,NaN,NaN,NaN,NaN,0.566897,NaN,NaN,NaN,NaN


In [65]:
df_genecl.to_csv('E:\\deeplearning\\Hepatocarcinomes\\TCGA\\processed_backup\\cophenet_genecl.csv', index=True, sep='\t')

In [67]:
print("row/gene cluster:")
distances_susp = distances[:-3]
df_genecl_susp = pd.DataFrame(columns=distances_susp)
for link in linkages:
    print(link)
    list_cophenet = []
    for dis in distances_susp:
        dist = distance.pdist(data_full_transformed_samplecl.values, metric=dis) # sqeuclidean
        cophenet_dist = cophenet(linkage(y=dist, method=link), dist)[0]
        list_cophenet.append(cophenet_dist)
        print(dis, cophenet_dist) # the best is chebyshev 0.90
    df_genecl_susp = pd.concat([df_genecl_susp, pd.DataFrame(list_cophenet, columns=[link], index=distances_susp).T])
    print()
print("ward")
dist = distance.pdist(data_full_transformed_samplecl.values, metric="euclidean")
cophenet_dist = cophenet(linkage(dist, "ward"), dist)[0]
print("euclidean", cophenet_dist)
df_genecl_susp = pd.concat([df_genecl_susp, pd.DataFrame([cophenet_dist], columns=["euclidean"], index=["ward"])])
display(df_genecl_susp)
df_genecl_susp.to_csv('E:\\deeplearning\\Hepatocarcinomes\\TCGA\\processed_backup\\cophenet_genecl_susp.csv', index=True, sep='\t')

row/gene cluster:
single
euclidean 0.5631971040705497
sqeuclidean 0.5131285818272071
cityblock 0.5827864829428633
cosine 0.5131285818272071
chebyshev 0.8713716232975721
braycurtis 0.47830150813578703

average
euclidean 0.750697581042358
sqeuclidean 0.7166784659766886
cityblock 0.7537902372743918
cosine 0.7166784659766885
chebyshev 0.9007654932993642
braycurtis 0.6834921867953436

complete
euclidean 0.5745902087583343
sqeuclidean 0.5412121251266496
cityblock 0.6194811428286091
cosine 0.5412036717617146
chebyshev 0.6874793085238805
braycurtis 0.5459582241234492

ward
euclidean 0.333709933276969


C:\Program Files\Anaconda3\envs\r-tensorflow\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,braycurtis,chebyshev,cityblock,cosine,euclidean,sqeuclidean
single,0.478302,0.871372,0.582786,0.513129,0.563197,0.513129
average,0.683492,0.900765,0.753790,0.716678,0.750698,0.716678
complete,0.545958,0.687479,0.619481,0.541204,0.574590,0.541212
ward,NaN,NaN,NaN,NaN,0.333710,NaN
